# DSatur algorithm

## References

[1] Brélaz, D., 1979. New methods to color the vertices of a graph. *Communications of the ACM*, 22(4), 251-256.

[2] https://github.com/bruscalia/optimization-demo-files/blob/1fa7a3825421d0b166195d890f2629c576cfbfda/graph-coloring/graph_coloring.ipynb

## Model Formulation

### Sets and Indices

$i,j \in V=\{ 0, \ldots, n-1 \}$: Indices and set of vertices.

### Parameters

$n \in \mathbb{N}$: number of nodes.

$(i,j) \in E \subseteq V^2$: Set of edges.

### Decision Variables

$c_{i} \in \{ 0, \ldots, n-1 \}$: Color of node $i \in V$.

### Objective Function

- **Number of colors**. We want to minimize the number of colors.

\begin{equation}
\min Z=\max_{i \in \{ 0, \ldots, n-1 \}} c_i
\tag{0}
\end{equation}

### Constraints

- **Adjacent vertices**. No two adjacent vertices have the same color:

\begin{equation}
c_i \neq c_j \forall (i,j) \in E
\tag{1}
\end{equation}

# Python implementation

## Import the libraries

The following code imports the required libraries.

In [6]:
from typing import List, Tuple
import pandas as pd

ModuleNotFoundError: No module named 'ortools'

## Definitions

### Color and node

In [ ]:
class Color:

    index: int
    n_nodes: int

    def __init__(self, index) -> None:
        self.index = index
        self.n_nodes = 0

    def __repr__(self):
        return f"C{self.index}"

    def add_node(self):
        self.n_nodes = self.n_nodes + 1

class Node:

    neighbors: List['Node']
    index: int
    color: Color

    def __init__(self, index):
        self.index = index
        self.neighbors = []
        self.color = None

    def __repr__(self) -> str:
        return f"N{self.index}|{self.color}"

    def add_neighbor(self, node: 'Node'):
        if node not in self.neighbors:
            self.neighbors.append(node)

    def set_color(self, color: Color):
        self.color = color
        color.add_node()

    @property
    def neighbor_colors(self):
        return [n.color for n in self.neighbors if n.color is not None]

    @property
    def saturation(self):
        return len(set((n.color for n in self.neighbors if n.color is not None)))

    @property
    def degree(self):
        return len(self.neighbors)

### DSatur

In [ ]:
class DSatur:

    N: List[Node]
    C: List[Color]
    history: List[Node]

    def __init__(self, nodes: List[int], edges: List[Tuple[int, int]]):
        """Graph Coloring DSatur Algorithm proposed by Brélaz (1979)

        Brélaz, D., 1979. New methods to color the vertices of a graph.
        Communications of the ACM, 22(4), 251-256.

        Parameters
        ----------
        nodes : List[int]
            List of node indexes for which colors should be defined

        edges : List[Tuple[int, int]]
            List of edges for which nodes can't be assigned to the same color
        """
        N = [Node(i) for i in nodes]
        for e in edges:
            i, j = e
            N[i].add_neighbor(N[j])
            N[j].add_neighbor(N[i])
        self.N = N
        self.C = []
        self.history = []

    def find_next_color(self, node: Node) -> Color:
        """Finds the next available color to assign to a given node

        Parameters
        ----------
        node : Node
            Node considered in step

        Returns
        -------
        Color
            Next color available
        """
        next_color = None
        for c in self.C:
            if c not in node.neighbor_colors:
                next_color = c
                break
        if next_color is None:
            next_color = Color(len(self.C) + 1)
            self.C.append(next_color)
        return next_color

    def solve(self, save_history=False):
        """Solve the instance

        Parameters
        ----------
        save_history : bool, optional
            Either or not to store a sequence of colores nodes in the `history` attribute,
            by default False
        """
        Q = [n for n in self.N]  # Pool of uncolored nodes
        while len(Q) > 0:
            Q.sort(key=lambda x: (x.saturation, x.degree), reverse=True)
            n: Node = Q.pop(0)
            next_color = self.find_next_color(n)
            n.set_color(next_color)
            if save_history:
                self.history.append(n)
        self.C.sort(key=lambda x: x.n_nodes, reverse=True)

    @property
    def cost(self):
        return len(self.C)

## Create the data

The code below creates the data for the problem.  

### Read the file

In [ ]:
url = 'https://raw.githubusercontent.com/jacubero/Optimization/main/coloring/data/gc_50_3'
df = pd.read_csv(url, sep=" ", header=None)
df.head()

### Search

In [ ]:
node_count = int(df.at[0,0])
edge_count = int(df.at[0,1])

print("Number of nodes =", node_count)
print("Number of edges =", edge_count)

edges = []
node_set = set()
for i in range(1, edge_count + 1):
    edges.append((int(df.at[i,0]), int(df.at[i,1])))
    node_set.add(int(df.at[i,0]))
    node_set.add(int(df.at[i,1]))

nodes = sorted(node_set)
assert len(nodes) == node_count, "Wrong number of nodes specified"

dsatur = DSatur(nodes, edges)
dsatur.solve(save_history=True)

solution = []
for node in dsatur.N:
    solution.append(int(str(node).split("|")[1][1:])-1)

solution_node_set = set(solution)
num_colors = len(solution_node_set)


## Prints the solution

Prints the solution in the specified output format

In [ ]:
# prepare the solution in the specified output format
output_data = str(num_colors) + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, solution))

print(output_data)

## Visualize the solution

### Import libraries

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

### Generate colors used in nodes

In [ ]:
def generate_colors(num_colors):
    cmap = plt.get_cmap('tab20')  # You can choose a different colormap
    colors = [cmap(i) for i in np.linspace(0, 1, num_colors)]
    return colors

### Create graph 

In [ ]:
node_list = range(node_count)
edge_list = []
for i in range(1, edge_count + 1):
    edge_list.append((df.at[i,0], df.at[i,1]))

# Create a networkx graph from the edges
G = nx.Graph()
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

### Plot graph

In [ ]:
plot_colors = generate_colors(num_colors)

node_color_list = [plot_colors[color] for color in solution]

# Draw the graph with node colors
pos = nx.spring_layout(G)  # You can use different layout algorithms

# Draw nodes with specified colors
nx.draw(G, pos, with_labels=True, node_size=700, node_color=node_color_list)

# Display the graph
plt.show()

# Shell script execution

## Upload data

In [ ]:
%%shell

wget -nc -P ./data https://raw.githubusercontent.com/jacubero/Optimization/main/coloring/data/data.zip
cd data
unzip data.zip
rm data.zip
ls

## Execute solver

In [ ]:
%%shell

wget -nc https://raw.githubusercontent.com/jacubero/Optimization/main/coloring/dsatur/solver.py

mkdir -p dsatur

for file in $(ls ./data/*)
do
  filename="$(basename "$file")"
  python3 solver.py $file > ./dsatur/$filename.dsa
done